In [33]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup
import pandas as pd
import requests
import time

In [34]:
# driver = webdriver.Chrome()
# driver.get("https://www.ewg.org/tapwater/")

In [35]:
zip_csv = '../Resources/Data/usa_zipcode_of_or-1554j.csv'
zip_df = pd.read_csv(zip_csv)

In [36]:
zip_df.dropna()

,zip,Zipcode name,City,State,County Name
0,97810,"ADAMS, OR",ADAMS,OR,UMATILLA
1,97901,"ADRIAN, OR",ADRIAN,OR,MALHEUR
2,97406,"AGNESS, OR",AGNESS,OR,CURRY
3,97321,"ALBANY, OR",ALBANY,OR,LINN
4,97322,"ALBANY, OR",ALBANY,OR,LINN
...,...,...,...,...,...
457,97497,"WOLF CREEK, OR",WOLF CREEK,OR,JOSEPHINE
458,97071,"WOODBURN, OR",WOODBURN,OR,MARION
459,97498,"YACHATS, OR",YACHATS,OR,LINCOLN
460,97148,"YAMHILL, OR",YAMHILL,OR,YAMHILL


In [37]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [38]:
zip_list = zip_df.zip.to_list()
# del zip_list[-3:]
# for zip_group in chunker(zip_list,5):
#     print(zip_group)

In [39]:
# zip_list = zip_df.zip.to_list()
zip_list.index('97002')
# zip_list =zip_list[:10]
zip_list = zip_list[16:]
zip_list

['97002',
 '97410',
 '97814',
 '97411',
 '97106',
 '97817',
 '97107',
 '97621',
 '97108',
 '97004',
 '97003',
 '97005',
 '97006',
 '97007',
 '97008',
 '97075',
 '97076',
 '97077',
 '97078',
 '97701',
 '97702',
 '97707',
 '97708',
 '97709',
 '97412',
 '97326',
 '97413',
 '97622',
 '97818',
 '97623',
 '97009',
 '97010',
 '97819',
 '97011',
 '97414',
 '97903',
 '97415',
 '97327',
 '97720',
 '97522',
 '97109',
 '97416',
 '97730',
 '97013',
 '97110',
 '97820',
 '97417',
 '97111',
 '97014',
 '97329',
 '97523',
 '97502',
 '97731',
 '97419',
 '97624',
 '97641',
 '97015',
 '97016',
 '97112',
 '97017',
 '97018',
 '97823',
 '97420',
 '97423',
 '97019',
 '97113',
 '97330',
 '97331',
 '97333',
 '97339',
 '97424',
 '97824',
 '97335',
 '97732',
 '97604',
 '97336',
 '97733',
 '97426',
 '97734',
 '97625',
 '97338',
 '97089',
 '97429',
 '97114',
 '97825',
 '97430',
 '97054',
 '97341',
 '97342',
 '97431',
 '97722',
 '97432',
 '97020',
 '97434',
 '97435',
 '97904',
 '97021',
 '97115',
 '97905',
 '97022',


In [40]:
#Initialize list to hold the scraped data   
contaminant_list = []

In [41]:
# driver = webdriver.Chrome()
# driver.get("https://www.ewg.org/tapwater/")
driver = webdriver.Chrome()
driver.get("https://www.ewg.org/tapwater/")

In [ ]:
for zip_group in chunker(zip_list,8):
    print(zip_group)
    driver.get("https://www.ewg.org/tapwater/")
    for zipcode in zip_group:
        print(zipcode)
        #try:
        #driver.manage().timeouts().pageLoadTimeout(40, TimeUnit.SECONDS)

        element = driver.find_element(By.XPATH,'/html/body/main/section[1]/form/input[1]')
        element.clear()
        element.send_keys(zipcode)
        button = driver.find_element(By.XPATH,'/html/body/main/section[1]/form/input[3]').click()
        zip_url = driver.current_url
        url = requests.get(zip_url)
        table = pd.read_html(url.text)
        num_utilities = len(table[0])

        #Initialize an empty list to contain all of our data
        for utility_vist in range(num_utilities):
            try:
                print(utility_vist)
                utility_vist = str(utility_vist+1)
                XPath = "/html/body/div[3]/main/figure[1]/table/tbody/tr["+utility_vist+"]/td[1]/a"
                selection = driver.find_element(By.XPATH, XPath).click()
                # Parse the resulting html with soup
                page_source = driver.page_source
                html_soup = soup(page_source, 'html.parser')

                #Get the name of the Water Utility
                Utility = html_soup.find('h1').text
                Utility

                #get the html data we need
                data_box = html_soup.find_all('div', class_='contaminant-name')

                for i in range(len(data_box)):
                    data = data_box[i].find_all('span')
                    data_measure = []
                    d = {
                        'Utility' : Utility,
                        'Contaminant': '', 
                        'Utility Measuremnt':'', 
                        'EWG HEALTH GUIDELINE': '',
                        'Legal Limit':'' 
                    }

                    contaminant_name = data_box[i].find('h3')
                    d['Contaminant'] = contaminant_name

                    for j in range(len(data)):
                        measurement = data[j].text
                        #print(measurement)
                        data_measure.append(measurement)
                        #print(data_measure)

                    try:
                        d['Utility Measuremnt'] = data_measure[data_measure.index('THIS UTILITY')+1]
                    except ValueError:
                        print("A value error arose")
                    except:
                        print("Something else went wrong")
                    try:
                        d['EWG HEALTH GUIDELINE'] = data_measure[data_measure.index('EWG HEALTH GUIDELINE')+1]
                    except ValueError:
                        print("A value error arose")
                    except:
                        print("Something else went wrong") 
                    try:
                        d['Legal Limit'] = data_measure[data_measure.index('LEGAL LIMIT')+1]
                    except ValueError:
                        print("A value error arose")
                    except:
                        print("Something else went wrong") 

                    contaminant_list.append(d)

                driver.back()
                time.sleep(5)

            except:
                pass

        #Navigate back to the home page after finishing the zip code scrape    
        driver.back()
        time.sleep(10)
        #except:
            #print('Level one error')
            #pass

['97002', '97410', '97814', '97411', '97106', '97817', '97107', '97621']
97002
0
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
1
2
3
4
5
6
7
8
9
97410
0
A value error arose
A value error arose
A value error arose
A value error arose
1
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
2
A value error arose
A value error arose
A value error arose
3
A value error arose
A value error arose
A value error arose
4
A value error arose
A value error arose
A value error arose
5
A value error arose
A value error arose
A value error arose
6
A value error arose
A value error arose
A value error arose
A value error arose
7
A value error arose
A value error arose
A value error arose
A value error arose

In [ ]:
df = pd.DataFrame(contaminant_list)

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

df

In [ ]:
driver.quit()

In [ ]:
df.to_csv('../Resources/Data/zips_scrape.csv')